# CSE 158 HW 3
#### 11/12/19
#### Hongtao Jiang A13760857
#### Kaggle User Name: Hongtao Jiang (Parker)
    Note: It shows as Hongtao Jiang on the leaderboard but the one in my profile is Parker

Before answering the questions, I imported all the models that will be used.

In [1]:
import pandas as pd
import numpy as np
import urllib
import scipy.optimize
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn import metrics
import gzip
from collections import defaultdict

Imported the two methods from the Baseline.py

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = pd.read_csv(path)
    f.readline()
    for l in f:
        yield l.strip().split(',')

## Question 1

First, I read the data and split it into training set and validation set. While I created two sets, I added a binary column to show if the reader has read the book. And then, I add 10000 negative entries to the validation sets. And save training set and validation set into two txt files. 

In [3]:
# read the file
f = gzip.open('train_Interactions.csv.gz','rt',encoding = 'utf8')
header = f.readline()
header = header.strip().split('\t')

# generate training set
train_data = []
for i in range(0,190000):
    l = f.readline().strip().split(',')
    train_data.append(l)
    
# generate validation set  
valid_data = []
for i in range(190001,200001):
    l = f.readline().strip().split(',')
    l.append('1')
    valid_data.append(l)

total_data = train_data+valid_data

# save training set to train_data.txt
with open('train_data.txt', 'w') as f:
    for i in train_data:
        f.write(i[0] +',' + i[1] +',' + i[2] + ',' + '1' + '\n')

# create a dictionary with key as users and 
# values as the books that the user has read        
bookUser_valid = defaultdict(list)
for i in valid_data:
    bookUser_valid[i[0]].append(i[1])

# create a list that contains all the books
book_lst = []
for i in total_data:
    book_lst.append(i[1])   
book_lst1 = list(set(book_lst))

user_lst = []
for i in total_data:
    user_lst.append(i[0])   
user_lst1 = list(set(user_lst))

# generate 10000 random negative entries for the validation set
sample = []
for i in valid_data:
    random_book = random.choice(book_lst1)
    while random_book in bookUser_valid[i[0]]:
        random_book = random.choice(book_lst1)
    sample.append([i[0],random_book,'0','0'])
    
valid_data = valid_data + sample

# save validation set to valid_data.txt
with open('valid_data.txt', 'w') as f:
    for i in valid_data:
        f.write(i[0] +',' + i[1] +',' + i[2] + ',' + i[3] + '\n')

Below code is modified baseline model from the baseline.py.

In [4]:
bookCount = defaultdict(int)
totalRead = 0

for l in open('train_data.txt'):
    u,book,r,i = l.strip().split(',') 
    bookCount[book]+=1
    totalRead+=1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

# create a correct variable to calculate the accuracy later        
correct = 0
predictions = open("predictions_Read.txt", 'w')
for l in open("valid_data.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b,r,i = l.strip().split(',')
    if b in return1:
        predictions.write(u + '-' + b + ",1\n")
        if i == '1':
            correct += 1
            
    else:
        predictions.write(u + '-' + b + ",0\n")
        if i == '0':
            correct += 1

predictions.close()

# calculate the accuracy score for baseline model
print('Accuracy Score:',correct/len(valid_data))

Accuracy Score: 0.643


## Question 2

To locate the threshold with the highest accuracy, I run the model multiple times with different threshold.

In [5]:
# create a list of threshold to test which one has the highest accuract
threshold = [0.1,0.2,0.3,0.4,0.5,0.566,0.7,0.8,0.9]

for j in threshold:

    return2 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return2.add(i)
        if count > totalRead*j: break

        
    correct = 0
    predictions = open("predictions_Read.txt", 'w')
    for l in open("valid_data.txt"):
        if l.startswith("userID"):
            #header
            predictions.write(l)
            continue
        u,b,r,i = l.strip().split(',')
        if b in return2:
            predictions.write(u + '-' + b + ",1\n")
            if i == '1':
                correct += 1
            
        else:
            predictions.write(u + '-' + b + ",0\n")
            if i == '0':
                correct += 1

    predictions.close()

    print('Accuracy Score with ' + str(j) + ' threshold:',correct/len(valid_data))

Accuracy Score with 0.1 threshold: 0.53975
Accuracy Score with 0.2 threshold: 0.5745
Accuracy Score with 0.3 threshold: 0.6072
Accuracy Score with 0.4 threshold: 0.6289
Accuracy Score with 0.5 threshold: 0.643
Accuracy Score with 0.566 threshold: 0.64885
Accuracy Score with 0.7 threshold: 0.641
Accuracy Score with 0.8 threshold: 0.617
Accuracy Score with 0.9 threshold: 0.578


It turns out that when the threshold is around 0.566, the accuracy is the highest, which is 0.64855.

## Question 3

Before calculate the Jaccard Similarity, I created two dictionary to avoid including too many loops in my method. The first dictionary's key is user and the values are the books that the user has read. The second dictionary's key is book and values are the users that have read the book.

In [6]:
bookUser_train = defaultdict(list)
for i in train_data:
    bookUser_train[i[0]].append(i[1])
    
userBook_train = defaultdict(list)
for i in train_data:
    userBook_train[i[1]].append(i[0])

I choose the threshold to be 0.01 so that the accuracy is optimized.

## Jaccard-User

In [7]:
# define jaccard similarity function
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

max_distance_lst = []
output_lst = []
for i in valid_data:
    # book that the user in the pair from validation set has read
    b_user = bookUser_train[i[0]]
    # users that have read the book in the pair in train data
    if i[1] in userBook_train:
        u_book = userBook_train[i[1]]
    else:
        u_book = []
    # for every book that is not the book in the pair, get all users for each book 
    u_book_xb = []
    for book in b_user:
        if book in userBook_train:
            u_book_xb += [userBook_train[book]]
    scores = [0]
    # compute the jaccard similarity list
    for user in u_book_xb:
        scores.append(jaccard_similarity(user,u_book))
    # if the maximum of the list exceeds the threshold, predict positive
    max_distance_lst.append(max(scores))
    if max(scores) > 0.01:
        output_lst.append('1')
    else:
        output_lst.append('0')

In [8]:
y = []
for i in valid_data:
    y.append(i[3])
print('Accuracy Score from Jaccard-User Similarity with a Threshold of 0.01:', metrics.accuracy_score(y, output_lst))

Accuracy Score from Jaccard-User Similarity with a Threshold of 0.01: 0.6202


## Jaccard-Book

In [9]:
max_distance_lst_bu = []
output_lst_bu = []
for i in valid_data:
    # users that the book in the pair from validation set has been read by them
    user_b = userBook_train[i[1]]
    # books that have been read by the user in the pair in train data
    if i[0] in bookUser_train:
        b_user = bookUser_train[i[0]]
    else:
        b_user = []
    # for every user that is not the user in the pair, get all books for each user 
    b_user_xb = []
    for user in user_b:
        if user in bookUser_train:
            b_user_xb += [bookUser_train[user]]
    scores_bu = [0]
    # compute the jaccard similarity list
    for book in b_user_xb:
        scores_bu.append(jaccard_similarity(book,b_user))
    # if the maximum of the list exceeds the threshold, predict positive
    max_distance_lst_bu.append(max(scores_bu))
    if max(scores_bu) > 0.03:
        output_lst_bu.append('1')
    else:
        output_lst_bu.append('0')
        

In [10]:
y = []
for i in valid_data:
    y.append(i[3])
print('Accuracy Score from Jaccard-Book Similarity with a Threshold of 0.03:', metrics.accuracy_score(y, output_lst_bu))

Accuracy Score from Jaccard-Book Similarity with a Threshold of 0.03: 0.6385


## Question 4

In order to combine two models, I add the jaccard similarity as a condition that the book must not only be among the popular ranking above 0.0566 threshold, but also has a jaccard similarity outcome of 1 (positive label) to be predicted as read by the user.

In [11]:
if_popular = []

return4 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return4.add(i)
    if count > totalRead*0.566: break
        
for i in valid_data:
    if i[1] in return4:
        if_popular.append(1)
    else:
        if_popular.append(0)

index = 0
correct = 0
for l in open("valid_data.txt"):
    u,b,r,i = l.strip().split(',')
    # add the output list from jaccard similarity as a condition
    if b in return4 and output_lst[index] == '1':
        if i == '1':
            correct += 1            
    else:
        if i == '0':
            correct += 1
    index += 1


print('Accuracy Score for the combined model:',correct/len(valid_data))   

Accuracy Score for the combined model: 0.6499


## Question 5

#### Kaggle User Name: Hongtao Jiang (Parker)
    Note: It shows as Hongtao Jiang on the leaderboard but the one in my profile is Parker

Before running the model on testing set, we need to re-train out jaccard similarity model so that it will perform well on the testing set.

In [12]:
# read test data
test_data = []
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        #header
        continue
    u,b = l.strip().split('-')
    test_data.append([u,b])

Here, I defined two functions to compute jaccard similarity. 

The first one is *compute_jaccard_max*, which returns the binary based on if the maximum of each similarity list exceed the threshold. 

The second one is *compute_jaccard_mean*, which returns the binary based on if the mean of each similarity list exceed the threshold.

Then, to combine the popularity and the jaccard similarity, I first check if the book in the pair is in the popularity rank above the threshold, then if it is, I set an index to 1. Then I sum the index with three different jaccard similarity computation (First two with compute_jaccard_max with different threshold parameter, and the last one with compute_jaccard_mean) to see if the sum exceeds 2. If it does, I predict the outcome to be positive. Otherwise, the prediction is negative.

In [13]:
def cos_similarity(a,b):
    a=set(a)
    b=set(b)
    numer = len(a.intersection(b))
    if len(a)==0 or len(b)==0:
        return 0
    cos = numer /(pow(len(a),1/2)*pow(len(b),1/2))
    return cos

def compute_cosine_max(data,threshhold=0.02):
    output_lst = []
    for i in data:
        # book that the user in the pair from validation set has read
        b_user = bookUser_train[i[0]]
        # users that have read the book in the pair in train data
        if i[1] in userBook_train:
            u_book = userBook_train[i[1]]
        else:
            u_book = []
        # for every book that is not the book in the pair, get all users for each book 
        u_book_xb = []
        for book in b_user:
            if book in userBook_train:
                u_book_xb += [userBook_train[book]]
        scores = [0]
        # compute
        for user in u_book_xb:
            scores.append(cos_similarity(user,u_book))
        if max(scores) > threshhold:
            output_lst.append(1)
        else:
            output_lst.append(0)
    return output_lst

In [21]:
l=compute_cosine_max(valid_data, threshhold=0.025)
y = []
acc=0
for i in valid_data:
    y.append(i[3])
for i in range(len(y)):
    #print("i")
    if int(y[i])==l[i]:
        acc+=1
print(acc/len(y))

0.59975


In [23]:
def compute_jaccard_max(threshhold=0.01):
    output_lst = []
    for i in test_data:
        # book that the user in the pair from validation set has read
        b_user = bookUser_train[i[0]]
        # users that have read the book in the pair in train data
        if i[1] in userBook_train:
            u_book = userBook_train[i[1]]
        else:
            u_book = []
        # for every book that is not the book in the pair, get all users for each book 
        u_book_xb = []
        for book in b_user:
            if book in userBook_train:
                u_book_xb += [userBook_train[book]]
        scores = [0]
        # compute
        for user in u_book_xb:
            scores.append(jaccard_similarity(user,u_book))
        if max(scores) > threshhold:
            output_lst.append(1)
        else:
            output_lst.append(0)
    return output_lst

def compute_jaccard_max_2(threshhold=0.03):
    output_lst_bu = []
    for i in test_data:
        # users that the book in the pair from validation set has been read by them
        user_b = userBook_train[i[1]]
        # books that have been read by the user in the pair in train data
        if i[0] in bookUser_train:
            b_user = bookUser_train[i[0]]
        else:
            b_user = []
        # for every user that is not the user in the pair, get all books for each user 
        b_user_xb = []
        for user in user_b:
            if user in bookUser_train:
                b_user_xb += [bookUser_train[user]]
        scores_bu = [0]
        # compute the jaccard similarity list
        for book in b_user_xb:
            scores_bu.append(jaccard_similarity(book,b_user))
        # if the maximum of the list exceeds the threshold, predict positive
        if max(scores_bu) > threshhold:
            output_lst_bu.append(1)
        else:
            output_lst_bu.append(0)
    return output_lst_bu
        

def compute_jaccard_mean(threshhold=0.001):
    output_lst = []
    for i in test_data:
        # book that the user in the pair from validation set has read
        b_user = bookUser_train[i[0]]
        # users that have read the book in the pair in train data
        if i[1] in userBook_train:
            u_book = userBook_train[i[1]]
        else:
            u_book = []
        # for every book that is not the book in the pair, get all users for each book 
        u_book_xb = []
        for book in b_user:
            if book in userBook_train:
                u_book_xb += [userBook_train[book]]
        scores = [0]
        # compute
        for user in u_book_xb:
            scores.append(jaccard_similarity(user,u_book))
        if np.mean(scores) > threshhold:
            output_lst.append(1)
        else:
            output_lst.append(0)
    return output_lst

# create three jaccard similarity lists
jaccard_lst1 = compute_jaccard_max(threshhold=0.01)
jaccard_lst2 = compute_jaccard_max(threshhold=0.05)
jaccard_lst3 = compute_jaccard_mean(threshhold=0.001)
jaccard_lst4 = compute_jaccard_max_2(threshhold=0.03)
#cosine_lst = compute_cosine_max(test_data, threshhold=0.028)

#cosine_lst1=compute_cosine_max(test_data,threshhold=0.028)

index = 0
predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    t = 0
    # first check if the book is in the popularity rank
    if b in return4:
        t = 1
    # then check if the sum of three jaccard scores and the index is above 2
    if t + jaccard_lst1[index] + jaccard_lst2[index] + jaccard_lst3[index] + jaccard_lst4[index] >= 3:
        predictions.write(u + '-' + b + ",1\n")
    else:
        predictions.write(u + '-' + b + ",0\n")
    index += 1
predictions.close()

My accuracy on Kaggle is 0.66933, which has improved.

## Question 6

First, read the file and split it into training set and validation set.

For conviniences, I transform each element of the training and validation set into a dictionary from a string of dictionary.

In [17]:
import ast

f = readGz('train_Category.json.gz')
#header = f.readline()
#header = header.strip().split('\t')


train_data = []
valid_data = []
data = []

index = 0

for line in f:
    data.append(line)
    if index < 190000:
        train_data.append(line)
    else:
        valid_data.append(line)
    index += 1
    

Then, I computed the 10 most common words with their frequencies.

In [26]:
import string
import nltk

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in train_data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]

print("10 most recent words are:")
for i in range(10):
    print(counts[i])

10 most recent words are:
(1421430, 'the')
(858918, 'and')
(754129, 'a')
(716863, 'to')
(699878, 'i')
(622766, 'of')
(420605, 'is')
(408519, 'in')
(392647, 'it')
(370479, 'this')


## Question 7

For this question, I used the feature function from class code, and apply it onto training set to generate feature vector. Then, I trained a logistic regression on the training dataset, and then test its accuracy on the validation set.

In [21]:
# Sentiment analysis
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)
punctuation = set(string.punctuation)

# feature function from class note
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

# generate training x and y
X_train = [feature(d) for d in train_data]
y_train = [d['genreID'] for d in train_data]

# generate validation x and y
X_valid = [feature(d) for d in valid_data]
y_valid = [d['genreID'] for d in valid_data]

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train,y_train)

# calculate the accuracy
print('Accuracy Score from the Logistic Regression:', metrics.accuracy_score(clf.predict(X_valid), y_valid))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy Score from the Logistic Regression: 0.6435


## Question 8

First, I change the dictionary size to 2000 to increase my accuracy. 

In [22]:
words_8 = [x[1] for x in counts[:2000]]

Then, I go through the same process as question 7 on the new word dictionary.

In [23]:
# Sentiment analysis
wordId_8 = dict(zip(words_8, range(len(words_8))))
wordSet_8 = set(words_8)
punctuation = set(string.punctuation)

def feature_8(datum):
    feat = [0]*len(words_8)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in words_8:
            feat[wordId_8[w]] += 1
    feat.append(1) #offset
    return feat

X_train_8 = [feature_8(d) for d in train_data]
y_train_8 = [d['genreID'] for d in train_data]

clf_8 = LogisticRegression()
clf_8.fit(X_train_8,y_train_8)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

After I got the new logistic model, I fit it on the testing set and then save it to the pairs_Category.txt.

In [25]:
# read testing data set
f = readGz('test_Category.json.gz')
# convert the testing set in a dataframe for further use
X_test = pd.DataFrame(f)
# generate x from testing set
X_test = X_test.apply(feature_8,axis = 1)
X_test = pd.DataFrame.from_items(zip(X_test.index, X_test.values)).T
data = pd.read_csv('pairs_Category.txt', sep=",", header=0)
# use the trained model on the testing set
data['prediction'] = clf_8.predict(X_test)
# save the result to pairs_Category.txt
data.to_csv('pairs_Category.txt', sep=',', index=False)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  import sys


My accuracy on Kaggle is 0.70566, which has improved and identical with the solution.